# **LoopsResultsExploration**

## **1. Importing and First Proccesing**

In [1]:
import pandas as pd
import plotly.express as px
from DataCleaning import *
from ProcessingConfig import *
from scipy import stats

pd.options.display.max_columns = 50

In [2]:
raw_data = pd.read_excel(cleaning_config['raw_data_path'])

raw_data = pd.read_excel(cleaning_config['raw_data_path'])
print(f'original shape: {raw_data.shape}')
outliers_threshold = cleaning_config['filter_threshold']
print(f"threshold for outliers detection: {outliers_threshold}")

drop_columns(raw_data, cleaning_config['unnecessary_columns'])
convert_types(raw_data, cleaning_config['type_conversions'])
raw_data = filter_slow_subjects(raw_data, outliers_threshold)
raw_data = filter_bad_subjects(raw_data, outliers_threshold)
raw_data = drop_first_loop(raw_data)
raw_data = only_first_line(raw_data)
# raw_data = filter_bad_trials(raw_data, threshold=analysis_config['trials_success_rate_threshold'])
# raw_data = filter_slow_steps(raw_data, outliers_threshold)

print(f'final shape: {raw_data.shape}')

original shape: (17408, 32)
threshold for outliers detection: 2.25
filter_slow_subjects: No slow subjects detected.
filter_bad_subjects: No bad subjects detected (in terms of low success rate).
drop_first_loop: 2134 rows were filtered out.
only_first_line: 12081 rows were filtered out.
final shape: (3193, 25)


In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 6 to 17404
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   subject                       3193 non-null   object 
 1   step_num                      3193 non-null   int64  
 2   uid                           3193 non-null   int64  
 3   trial_start_time              3193 non-null   int64  
 4   rt                            3193 non-null   int32  
 5   response                      3193 non-null   object 
 6   loop_step                     3193 non-null   int32  
 7   trial_order                   3193 non-null   int64  
 8   trial_set                     3193 non-null   int64  
 9   trial                         3193 non-null   int64  
 10  core_program                  3193 non-null   int64  
 11  variant                       3193 non-null   object 
 12  step_id                       3193 non-null   int64  
 13  te

In [4]:
raw_data.sample(3)

,subject,step_num,uid,trial_start_time,rt,response,loop_step,trial_order,trial_set,trial,core_program,variant,step_id,text1,response_needed,expected_response,is_loop,loop_type,loop_type_switch,n_iterations,n_loop_lines,expected_response_whole_loop,step_type,prev_loop_type,correct
12768,108A,392,14,1218455,1979,9,0,11,1,2,1,FWWWWFFW,5,"while a >= 7:\n a = average(a,5)",True,9.0,True,while,False,3,1,"[9, 7, 6]",loop,while,True
7980,105B,182,44,635624,3905,6,0,5,1,5,3,WFFFWWWF,8,while a >= 12:\n a /= 3\n a *= 2,True,6.0,True,while,False,2,2,"[6, 12, 4, 8]",loop,while,True
10921,107A,375,108,956177,1798,6,0,10,2,12,6,FFWWFWWF,9,"for i in [1, 2]:\n a -= 5",True,6.0,True,for,True,2,1,"[6, 1]",loop,while,True


In [5]:
raw_data['subject'].sort_values().unique()

array(['101A', '101B', '102A', '102B', '103A', '103B', '104A', '104B',
       '105A', '105B', '106A', '106B', '107A', '107B', '108A', '108B',
       '109A', '109B', '110A', '110B', '111A'], dtype=object)

## **2. Exploring Outliers**

### 2.1. Response-Time Between Subjects

In [6]:
# filtering only necessary columns
response_times = raw_data[['subject', 'step_num', 'rt']].copy()

In [7]:
# checking for outliers in terms of mean response time
mean_rt_per_subject = response_times[['rt', 'subject']].groupby('subject').mean()
mean_rt_per_subject.columns = ['mean_rt']

g_rt_q1, g_rt_q3 = mean_rt_per_subject['mean_rt'].quantile([0.25, 0.75])
g_rt_iqr = g_rt_q1 - g_rt_q3

mean_outlier_mask = mean_rt_per_subject['mean_rt'].apply(is_negative_outlier
                                                      , args=(g_rt_q1, g_rt_iqr
                                                              , cleaning_config['filter_threshold']))
rt_outlier_subject = mean_rt_per_subject[mean_outlier_mask].index

if rt_outlier_subject.size > 0:
    print(f'Seems that: {list(rt_outlier_subject)} are outliers in terms of mean response time within subject.')
else:
    print("No Outliers detected! at least in terms of mean response time within subject.")

No Outliers detected! at least in terms of mean response time within subject.


### 2.2. Mistakes Rate (%) Between Subjects

In [18]:
# filtering only necessary columns
response_success = raw_data[['subject', 'step_num', 'trial', 'correct', 'loop_step', 'loop_type_switch']].copy()

# calculating quantiles
g_c_mean = response_success['correct'].mean()
print(f'mean general success rate: {round(g_c_mean, 2)}')

mean general success rate: 0.96


In [19]:
success_per_subject = response_success[['subject', 'correct']].groupby('subject').mean()
success_per_subject.rename(columns={'correct': 'success_rate'}, inplace=True)

# manually excluding perfect 0 success rate subjects
success_per_subject = success_per_subject[success_per_subject['success_rate'] > 0]

success_per_subject.sort_values(by='success_rate', ascending=False).T

subject,107A,103B,107B,108A,103A,105A,106B,104B,109B,111A,108B,104A,106A,102B,101A,105B,110B,109A,102A,110A,101B
success_rate,1.0,1.0,1.0,0.994048,0.988095,0.988095,0.988095,0.982143,0.97619,0.970238,0.964286,0.964286,0.952381,0.952381,0.946429,0.928571,0.928571,0.904762,0.904762,0.880952,0.87574


In [20]:
success_rate_q1, success_rate_q3 = success_per_subject['success_rate'].quantile([0.25, 0.75])
success_rate_iqr = success_rate_q3 - success_rate_q1

success_outlier_mask = success_per_subject['success_rate'].apply(is_negative_outlier
                                        , args=(success_rate_q1, success_rate_iqr
                                            , cleaning_config['filter_threshold']))

success_outlier_subjects = success_per_subject[success_outlier_mask]

if success_outlier_subjects.size > 0:
    print(f'Seems that: {list(success_outlier_subjects.index)} are outliers in terms of success rate within subject.')
else:
    print("No Outliers detected! at least in terms of success rate within subject.")

No Outliers detected! at least in terms of success rate within subject.


### 2.3. Trial Mistakes Rate (%)

In [21]:
# calculating response success rate per trial
success_per_trial = response_success[['subject', 'trial', 'correct']].groupby(['subject', 'trial']).mean()
success_per_trial.rename(columns={'correct': 'success_per_trial'}, inplace=True)

# manually excluding perfect 0 success rate trials
success_per_trial = success_per_trial[success_per_trial['success_per_trial'] > 0]

success_per_trial

success_per_trial
subject trial                   
101A    1               1.000000
        2               1.000000
        3               0.857143
        4               0.857143
        5               0.857143
...                          ...
111A    1008            1.000000
        1009            1.000000
        1010            1.000000
        1011            1.000000
        1012            1.000000

[456 rows x 1 columns]

In [22]:
pr_trials_hist = px.histogram(success_per_trial, nbins=20, x="success_per_trial"
                   , title='Success Rate per Trial (only first steps)', labels={'success_per_trial':'success rate'})
pr_trials_hist.show()

In [23]:
subjects =  list(set([sub for sub, trial in success_per_trial.index]))
subjects.sort()

# calculating std success rate per trial
trials_success_std = {'subject': [], 'success_per_trials_std': []}
for subject in subjects:
    trials_success_std['subject'].append(subject)
    try: 
        success_std = success_per_trial.loc[subject, 'success_per_trial'].std()
        success_std = round(success_std, 5)
        trials_success_std['success_per_trials_std'].append(success_std)
    except:
        trials_success_std['success_per_trials_std'].append(0.0)
  
success_per_trials_std = pd.DataFrame(trials_success_std)

# plotting the std success rates per trial in a histogram
trials_success_std_hist = px.histogram(success_per_trials_std, nbins=10, x="success_per_trials_std"
                   , title='STD Success Rate per Trial (only first steps)', labels={'success_per_trials_std':'success rate'})
trials_success_std_hist.show()

In [24]:
i = success_per_trials_std['success_per_trials_std'].idxmax()
subject_i = success_per_trials_std.loc[i, 'subject']
trials_success_within_subject_hist = px.histogram(success_per_trial.loc[subject_i, 'success_per_trial'], nbins=10, x="success_per_trial"
                   , title=f'Success Rate per Trial {subject_i}', labels={'success_per_trial':'success rate'})
trials_success_within_subject_hist.show()


In [25]:
switching_diff_per_trial_success = pd.DataFrame()

# merging success_per_trial with raw_data, and excluding perfect 0 success rate trials
success_per_trial_diff = success_per_trial.reset_index()
success_per_trial_diff = raw_data.merge(success_per_trial_diff, how='left', on=['subject', 'trial'])
success_per_trial_diff = success_per_trial_diff[success_per_trial_diff['success_per_trial'] > 0]
success_per_trial_diff.rename(columns={'correct_x': 'correct', 'loop_type_switch_x': 'loop_type_switch'
                                       , 'rt_x': 'rt', 'step_id_x': 'step_id'}
                              , inplace=True)

# mean response time and success rate, group by loop switching and success rate per trial.
success_per_trial_diff['success_per_trial_bin'] = pd.cut(success_per_trial_diff['success_per_trial'], bins=3).astype(str)
success_per_trial_diff['success_per_trial_bin'].unique()
switching_diff_per_trial_success['mean_response_time'] = success_per_trial_diff[success_per_trial_diff['correct']].groupby(['success_per_trial_bin', 'loop_type_switch'])['rt'].mean()
switching_diff_per_trial_success['size'] = success_per_trial_diff[success_per_trial_diff['correct']].groupby(['success_per_trial_bin', 'loop_type_switch'])['step_id'].count()

switching_diff_per_trial_success.unstack()

mean_response_time               size      
loop_type_switch                   False        True  False True 
success_per_trial_bin                                            
(0.714, 0.81]                3637.071429  3534.680000    56    50
(0.81, 0.905]                3505.984733  3631.304511   262   266
(0.905, 1.0]                 3608.888980  3705.128607  1216  1213

In [26]:
px.bar(switching_diff_per_trial_success.reset_index(), x='success_per_trial_bin', y='mean_response_time', color='loop_type_switch', barmode='group')

### Worse performance per trial in terms of success rate, seems to slightly reverse the priming effect.

In [36]:
success_per_trial_non_zero = success_per_trial[success_per_trial['success_per_trial'] > 0]
priming_effect_size_per_trial = raw_data.groupby(['subject', 'trial', 'loop_type_switch'])['rt'].mean().unstack()
priming_effect_size_per_trial['priming_effect_size'] = (priming_effect_size_per_trial[True] - priming_effect_size_per_trial[False]) / priming_effect_size_per_trial[False]
priming_effect_size_per_trial = priming_effect_size_per_trial.merge(success_per_trial, how='left', left_index=True, right_on=['subject', 'trial'])
priming_effect_size_per_trial.reset_index(inplace=True)
success_rate_and_priming_effect = priming_effect_size_per_trial[['subject', 'trial', 'success_per_trial', 'priming_effect_size']].drop_duplicates()

stats.pearsonr(success_rate_and_priming_effect['success_per_trial'], success_rate_and_priming_effect['priming_effect_size'])

PearsonRResult(statistic=-0.07550899447008863, pvalue=0.10733132284303251)

### As it seems, There is no significant correlation between success rate and priming effect size per trial.
##### priming effect size calculated as ```(switching_mean_rt - non_switching_mean_rt) / (non_switching_mean_rt)```

### 2.3. Single Steps Response-Time Within Subject

In [40]:
# calculating response time quantiles and IQR per subject
quantiles_per_subject = response_times[['rt', 'subject']].groupby('subject').quantile([0.25, 0.75]).unstack()
quantiles_per_subject.columns = ['q1', 'q3']
quantiles_per_subject['iqr'] = quantiles_per_subject['q3'] - quantiles_per_subject['q1']

quantiles_per_subject.head(3)

,q1,q3,iqr
subject,,,
101A,2569.0,5233.75,2664.75
101B,1282.0,2930.00,1648.00
102A,2552.5,5526.75,2974.25


In [41]:
# finding if a step is an outlier in terms of response time within subject
def is_subjective_outlier(step):
    rt = step['rt']
    subject = step['subject']
        
    subject_quantiles = quantiles_per_subject.loc[subject]
    q3, iqr = subject_quantiles['q3'], subject_quantiles['iqr']
    return is_positive_outlier(rt, q3, iqr, cleaning_config['filter_threshold'])

In [42]:
# finding outliers
subjective_outlier_mask = response_times.apply(is_subjective_outlier, axis=1)
outlier_steps = raw_data[subjective_outlier_mask]

# searching for outliers influence on effect
outlier_steps = outlier_steps[['subject', 'step_num', 'loop_type_switch', 'rt']]

outliers_and_effect = outlier_steps.groupby('loop_type_switch')['step_num'].count().to_frame()
outliers_and_effect.rename(columns={'step_num': 'num_of_steps'}, inplace=True)
outliers_and_effect['mean_rt'] = outlier_steps.groupby('loop_type_switch')['rt'].mean()

outliers_and_effect = outliers_and_effect.T
outliers_and_effect['diff'] = (outliers_and_effect[True] - outliers_and_effect[False]) / outliers_and_effect[False]
outliers_and_effect

loop_type_switch,False,True,diff
num_of_steps,41.000000,46.000000,0.121951
mean_rt,11318.097561,12588.043478,0.112205


In [43]:
def get_step_anomaly_grade(step):
    return abs(get_outlier_grade(step, 'rt'))

anomaly_grades = raw_data.merge(quantiles_per_subject, how='left', on='subject')
anomaly_grades['anomaly_grade'] = anomaly_grades.apply(get_step_anomaly_grade, axis=1)

px.histogram(anomaly_grades[anomaly_grades['anomaly_grade'] >=  cleaning_config['filter_threshold']], nbins=10
             , x='anomaly_grade', color='loop_type_switch', barmode='group',
             title=f'Outlier Steps Anomaly Grades', labels={'anomaly_grade':'Anomaly Grade (IQR)'})

### Priming effect size in outlier step is much bigger, but it seems it is a bias due to specific steps.

### 

## **3. Time per Session Analysis**

In [48]:
from os import listdir
from os.path import isfile, join

raw_results_folder = '../../msa-1/scripts/loops1/exp1_data/results/raw'

In [49]:
def get_total_times(path: str):
    # importing the file names from directory
    data_files = [f for f in listdir(path) if isfile(join(path, f)) and not f.startswith('Participant')]

    # creating a list of max time elapsed for every session
    max_times = {}
    for file_name in data_files:
        curr_times = pd.read_csv(path + f'\{file_name}', usecols=['time_elapsed'], dtype={'time_elapsed': int})
        curr_times = curr_times.squeeze()
        start_time = min(curr_times) / 60000
        end_time = max(curr_times) / 60000
        max_times.update({file_name :  (end_time - start_time)})
    
    return max_times

In [50]:
max_times = get_total_times(raw_results_folder)
only_times = list(max_times.values())

In [51]:
time_per_session = px.histogram(only_times, nbins=20, title='Time Per Session')
time_per_session.show()